# Analysis of outputs

In [ ]:
import os
import pandas as pd

## Read in `csvs`

In [ ]:
# path to output directory
dir = "../../output/"
# find csv files
filepaths = [f for f in os.listdir(dir) if f.endswith('.csv')]
# append the directory path to filename
filepaths_dir = [dir + filepath for filepath in filepaths]

# create empty list to append dataframes
dfs = []
for file in filepaths_dir:
    # read in files
    output=pd.read_csv(file)
    # Get the index date from the filename
    index_date = pd.to_datetime(file.split("_", )[1].split(".csv")[0], dayfirst=True)
    # Add the index date to the file
    output["index_date"] = index_date
    # Append the dataframes to the list
    dfs.append(output)

# Combine all the dataframes together
population_df = pd.concat(dfs)

In [ ]:
#population_df.head()

In [ ]:
#population_df.describe()

In [ ]:
#shielding_sum = pd.DataFrame(population_df.groupby("index_date")["shielding"].sum())

In [ ]:
# Create data frame of sums of each column for each index date
sum_df = population_df.groupby(['index_date'], as_index=False).sum()
#sum_df.head()

In [ ]:
# Change pulse oximetry headers in sum_df

# Convert pulse oximetry codelist csv into data frame
pulse_oximetry_codes_df = pd.read_csv('../../codelists/opensafely-pulse-oximetry.csv')
# Extract list of codes
pulse_oximetry_codes_list = pulse_oximetry_codes_df['code'].tolist()

# Create dictionary for renaming oximetry headers
oximetry_dictionary = {}
for n in range(0,len(pulse_oximetry_codes_df)):
    oximetry_dictionary[f'pulse_oximetry_{pulse_oximetry_codes_df.iat[n,0]}'] = pulse_oximetry_codes_df.iat[n,1]

# Rename oximetry headers in sums data frame
sum_df.rename(columns=oximetry_dictionary,inplace=True)

# Create list of headers
#pulse_oximetry_headers = [f'pulse_oximetry_{x}' for x in pulse_oximetry_codes_list]
pulse_oximetry_headers = pulse_oximetry_codes_df['term'].tolist()


In [ ]:
#sum_df.head()

In [ ]:
# Plot a time series of all of the oximetry codes
plot_1 = sum_df.plot.line('index_date',pulse_oximetry_headers, figsize=(20, 10), title = 'Use of Pulse Oximetry Codes Over Time', fontsize = 20)
plot_1.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0), fontsize = 20)
plot_1.set_xlabel("Date", fontsize = 20)
plot_1.title.set_size(40)